In [2]:
from medacy.model.model import Model
import pandas as pd

In [3]:
dat = pd.read_csv("../mtsample_scraped.csv")
notes = list(dat.medical_transcription)
nn = notes[0:10]

In [4]:
model = Model.load_external('medacy_model_clinical_notes')

In [9]:
annotation = []
for n in nn:
    ann = model.predict(n)
    annotation.append(list(ann))
annotation

[[('Drug', 196, 204, 'Claritin'),
  ('Drug', 210, 216, 'Zyrtec'),
  ('Drug', 1036, 1042, 'Zyrtec'),
  ('Drug', 1099, 1109, 'loratadine'),
  ('Drug', 1199, 1206, 'Nasonex'),
  ('Dosage', 1207, 1217, 'two sprays'),
  ('Route', 1218, 1233, 'in each nostril'),
  ('Duration', 1240, 1255, 'for three weeks')],
 [('Reason', 403, 420, 'perioral swelling'),
  ('ADE', 634, 650, 'allergy reaction'),
  ('Drug', 658, 664, 'Keflex'),
  ('Reason', 698, 738, 'skin cellulitis dialysis shunt infection'),
  ('Drug', 1002, 1010, 'atenolol'),
  ('Reason', 1015, 1037, 'blood pressure control'),
  ('Drug', 1124, 1138, 'corticosteroid'),
  ('Drug', 1151, 1164, 'antihistamine'),
  ('Drug', 1598, 1606, 'Atenolol'),
  ('Drug', 1608, 1626, 'sodium bicarbonate'),
  ('Drug', 1661, 1668, 'Heparin'),
  ('Reason', 3091, 3114, 'Acute allergic reaction'),
  ('Dosage', 3487, 3508, 'the sensitivity scale'),
  ('Drug', 3690, 3715, 'cephalosporin antibiotics'),
  ('Reason', 3747, 3764, 'allergic reaction'),
  ('Drug', 3813, 